#  Final Project - CSE 5334 - Data Mining

Name: Manthankumar Patel

Student ID: 1001778249


# The Goal of Project:

The purpose of this project is to build a model that will be trained on given data and will predict the ranting from 0 to 10 from the comment that we give.First, we will import the necessary libraries and the will load the data from csv file 'bgg-13m-reviews.csv'. Now, we will pre-process the data by formating the comments and rating and will divide into DataFrames and vectorize every DataFrames. I have used SVM classifier because it is a supervised machine learning algorithm that can be used for classification. In general terms SVM is very good when you have a huge number of features. For example for text classification in a bag of words model. 


# Importing Libraries and Loading Data:

First, we will import useful libraries and build csv path to load the data, which upon download is located in a directory called boardgamegeek-reviews, in csv file named ‘bgg-13m-reviews.csv’.

In [ ]:
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

from sys import path
from os.path import join

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Now, we will load the data, and using the pandas library we will read csv file. Then we will use only 'comment' and 'rating' and will replace NaN with ' '.

In [ ]:
def loadData(path):
    d = pd.read_csv(path)
    d = d[['comment', 'rating']].copy()
    d = d.fillna('')
    
    return d

# Formatting Data:

We have loaded data so we will do some preprocessing by formatting comments and ratings. From the comment, we have formatted white space and lower all the characters. From the rating, we have removed data that has 0 ratings and ensure that all ratings have values that are type float. 

In [ ]:
def formatRawData(rawData):
    rawData['comment'] = [c.strip().lower() for c in rawData['comment']]
    rawData['comment'] = [c if c.islower() else '' for c in rawData['comment']]
    
    #only take data with comments
    rawData = rawData[rawData['comment'].apply(lambda alpha: len(alpha) > 0)]
    #only take data with ratings 1 or greater
    rawData = rawData[rawData['rating'].apply(lambda alpha: float(alpha) >= 1)]
    rawData = rawData.sample(frac = 1).reset_index(drop = True)  
    
    rawData['rating'] = [float(rating) for rating in rawData['rating']]
    
    
    #for i in range(len(rawData)):
     #   words = re.split(r'\W+', rawData['comment'][i])
      #  stop_words = set(stopwords.words('english'))
       # words = [w for w in words if not w in stop_words]
        #rawData.loc[i, 'comment'] = ' '.join(map(str, words)).lower()
        
    return rawData

In [ ]:
rawData = loadData('/kaggle/input/boardgamegeek-reviews/bgg-13m-reviews.csv')

print(rawData['comment'][0])
print(len(rawData))

data = formatRawData(rawData)

print(data['comment'][0])
print(len(data))

 # Splitting Data:
 
Now we have to split dataframe into trainData, devData, testData and reset the index for every dataframe. From print below it can be seen that the train dataframe is close to 2100000 while development and test dataframe is nearly 9.5%  of all data. 

In [ ]:
def divideData(data, train):
    data = data.sample(frac=1).reset_index(drop = True)
    #dataLen = len(data['comment'])
    dataLen = data.shape[0]
    
    trainData = data[:int(train * dataLen)]
    devData = data[int(train * dataLen):int((train + 1)/2 * dataLen)]
    testData = data[int((train + 1)/2 * dataLen):]
    
    trainData = trainData.sample(frac = 1).reset_index(drop = True)
    devData = devData.sample(frac = 1).reset_index(drop = True)
    testData = testData.sample(frac = 1).reset_index(drop = True)
    
    return [trainData, devData, testData]

In [ ]:
trainData, devData, testData = divideData(data, 0.95).copy()

In [ ]:
print(trainData.shape[0])
print(devData.shape[0])
print(testData.shape[0])

# Vectorization:

As you can see above, I tried to remove stop words from comments but data size is too large so it took an enormously long time so I am using vectorization. Values in the comment are string so we need to convert every string into numerical series using vectorizer. Now we will copy and store every vectorized comment in train_c, test_c, dev_c respectively, and rating in train_r, test_r, dev_r.

In [ ]:
def vectorizeData(data, vectorizer):
    data_c = vectorizer.transform(data['comment'])
    data_r = np.asarray([int(r) if r%int(r) <= 0.5 else int(r + 1) for r in data['rating']])
    
    return [data_c, data_r]

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
vectorizer.fit(trainData['comment'])
train_c, train_r = vectorizeData(trainData, vectorizer).copy()
test_c, test_r = vectorizeData(testData, vectorizer).copy()
dev_c, dev_r = vectorizeData(devData, vectorizer).copy()

# SVM Classifier:

Finally, we have data within the appropriate format and that we are able to begin classifier.
Here I have used the SVM classifier because I researched on the internet that which classifier we can use for large data and I got on the conclusion that SVM is very good for text classification and  I have added the reference link bellow in References for this research. For example SVM with direct tf-idf vectors does the best both for quality & performance for text classification in a bag of words model.

# Hyperparameter Tuning:

Now, it is time to make some improvement in accuracy by hyperparameter tuning. we have to find the Regularization parameter-C to increase performance. we test for different C and show a relation between C and accuracy and we select the most effective value of C.

In [ ]:
#hyper parameter tuning 
C = [0.1, 0.3, 0.5, 0.7, 0.9, 1]

for c in C:
    svm = LinearSVC(C = c)
    svm.fit(train_c, train_r)
    
    pred_r = svm.predict(dev_c)

    accuracy = accuracy_score(dev_r, pred_r)*100

    print('SVM accuracy for C={}: {:.5f}%\n'.format(c, accuracy))
    

In [ ]:
svm = LinearSVC(C=0.1)
svm.fit(train_c, train_r)

In [ ]:
pred_r = svm.predict(dev_c)
accuracy = accuracy_score(dev_r, pred_r)*100
print('SVM accuracy for C=0.1: {:.5f}%\n'.format(accuracy))

In [ ]:
#Final Accuracy

predT_r = svm.predict(test_c)
accuracy = accuracy_score(test_r, predT_r)*100
print('SVM final accuracy: {:.5f}%\n'.format(accuracy))


Finally, That is a good improvement in accuracy over what SVM could do when it was only by itself. We build the  model that uses SVM and after that, we have done hyperparameter tuning to find optimal value and tested on test data and got good accuracy as you can see above.

# Conclusion and Challenges: 

In conclusion, we can say that by hyperparameter tuning we can increase accuracy in the SVM classifier. In addition, I learned, how vectorization is used to speed up the Python code without using a loop. 

In this project, there were many challenges because the data set was very huge and tough to predict that's why it took enormous time to execute vectorization and SVM. Plenty of the comments was very long and not all encoded within the same format. As for the ratings, class representation wasn't exactly even. To develop a model with such accuracy that would cope with such issues could likely only be finished by Neural Network. So many questions came in my mind such as-  Which classifier should I use? In which way I can increase performance? How to increase accuracy using hyperparameter tuning? Am I able to execute an optimization idea to extend accuracy? For all of those questions, I got answers.

Thank you for reading my blog post.

# References: 

Shuffle the data:
https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows

Vectorization:
https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

SVM:
https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html